# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条嵌入在翠绿草地中的木质步道。步道笔直延伸，目光可以随着路径向远方延伸，引人思考其通向何处。四周的草地颜色鲜绿，种类繁多的植被随风轻摇，彰显出自然的生机与活力。天空表现出晴朗的天气，云朵稀疏而飘逸，为这幅图增添了几分宁静与和谐的氛围。整体上，这幅图像给人一种平静、放松的感觉，是一处适合散步和沉思的理想场所。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图展示了一条嵌入在翠绿草地中的木质步道。步道笔直延伸，目光可以随着路径向远方延伸，引人思考其通向何处。四周的草地颜色鲜绿，种类繁多的植被随风轻摇，彰显出自然的生机与活力。天空表现出晴朗的天气，云朵稀疏而飘逸，为这幅图增添了几分宁静与和谐的氛围。整体上，这幅图像给人一种平静、放松的感觉，是一处适合散步和沉思的理想场所。'

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [4]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默的表现，通过将狗狗描绘成人体肌肉发达的形象来制造喜剧效果。图中左边是一只肌肉发达、站立的沙皮狗，被称为“16岁的我”，旁边的文字描述它强壮如斧头帮老大，是一种夸张和幽默的表现方式。右边是一只坐着的普通沙皮狗，被称为“工作后的我”，旁边有文字描述表达了工作压力大、身体状态差等笑话元素。这种对比通过夸张与现实的反差，表达了日常生活中人们对于理想与现实差距的幽默感受。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [5]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [6]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

该图展示了1980年至2020年间美国、中国、日本和德国的国内生产总值（GDP）的比较。图中的每条线代表一个国家的GDP增长轨迹（以万亿美元计）。

- 蓝色线代表美国，可以看到其GDP持续增长，特别是从1990年代中期开始加速上升。
- 红色线代表中国，显示其GDP在2000年前增长相对缓慢，但之后显著加速，尤其在2005年后增长速度非常快，直到2020年接近美国。
- 紫色线代表日本，GDP在1990年代初达到顶峰后略有下降，此后增长缓慢且波动，整体趋势相对平稳。
- 绿色线代表德国，GDP增长相对平稳，增速适中，无大幅波动。

总体上，这张图揭示了各国经济规模和增长速度的差异，其中中国的快速增长特别引人注目。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [7]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图是一本笔记本的手写页，记录了一些关于机器学习和人工智能中的“调优”（Tuning）技术的细节，特别是针对预训练模型如何进行修改以改善其性能的方法。具体内容如下：

1. **Prompt Tuning**：提到了Prompt Tuning技术用于小型模型。在这种策略中，输入X（一个包含序列\([X_1, X_2, ..., X_n]\)的向量）首先转换为embedding向量，然后与一组前缀token结合，从而产生新的序列输入。

2. **Prefix Tuning**：这是另一种调优技术，在这里参数W是增加到每个输入X前面的一串前缀 \(W_P\) 。增加的前缀参数与原有的变换器（Transformer）模型结合，即通过 \(W=[W_P; W]\) 形式，以此增加模型处理输入的灵活性。

3. **LoRA（Low-Rank Adaptation）技术**：与前两种技术不同，LoRA专注于调整网络的权重矩阵 \(W\)，通过引入一个低秩矩阵 \(\Delta W = AB\) 修改 \(W\) （其中 \(A\) 和 \(B\) 是新的可学习参数），以提高模型的表现。此外，还描述了具体的维度转换细节，如 \(\Delta W\) 的维度从 \(m \times n\) 到 \(m \times r\) 再到 \(r \times n\)。

最后，字迹处提到了两个关于LoRA技术的统计数据：LAMA 测试中LoRA使用了78GB，而QLoRA使用了48GB，指示不同配置下的资源使用情况。

总的来说，这页笔记总结了几种针对预训练深度学习模型的参数调整方法，以及相关技术的特点和应用场景。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [8]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图是一本笔记本的手写页，记录了一些关于机器学习和人工智能中的“调优”（Tuning）技术的细节，特别是针对预训练模型如何进行修改以改善其性能的方法。具体内容如下：

1. **Prompt Tuning**：提到了Prompt Tuning技术用于小型模型。在这种策略中，输入X（一个包含序列\([X_1, X_2, ..., X_n]\)的向量）首先转换为embedding向量，然后与一组前缀token结合，从而产生新的序列输入。

2. **Prefix Tuning**：这是另一种调优技术，在这里参数W是增加到每个输入X前面的一串前缀 \(W_P\) 。增加的前缀参数与原有的变换器（Transformer）模型结合，即通过 \(W=[W_P; W]\) 形式，以此增加模型处理输入的灵活性。

3. **LoRA（Low-Rank Adaptation）技术**：与前两种技术不同，LoRA专注于调整网络的权重矩阵 \(W\)，通过引入一个低秩矩阵 \(\Delta W = AB\) 修改 \(W\) （其中 \(A\) 和 \(B\) 是新的可学习参数），以提高模型的表现。此外，还描述了具体的维度转换细节，如 \(\Delta W\) 的维度从 \(m \times n\) 到 \(m \times r\) 再到 \(r \times n\)。

最后，字迹处提到了两个关于LoRA技术的统计数据：LAMA 测试中LoRA使用了78GB，而QLoRA使用了48GB，指示不同配置下的资源使用情况。

总的来说，这页笔记总结了几种针对预训练深度学习模型的参数调整方法，以及相关技术的特点和应用场景。

![](./images/handwriting_1.jpg)

In [9]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片显示的是一本笔记本的两页内容，记录了一些关于自然语言处理（NLP）和机器学习模型的信息。左页主要内容涉及变换器（Transformers）模型、PEFT、SOTA（State Of The Art）技术、PBFT方法以及不同类型的prompt tuning技巧，例如Adapter、Prefix、Prompt等。此外，还提到了一些机器学习模型如GPT、T5、BERT以及其他一些模型如LIAN、GPIXLHF、ChatGLM、BLOOM、Alpaca、MOSS等。

右页则涉及多模态的指令式调整技术（如Llama和LLMs），提到了LoRA和PETC等技术，以及Prefix-tuning和Adapters在大型语言模型（LLMs）中的应用，并探讨了插入型和非插入型调整技术，最后还提到了MAM Adapters与SOTA prompt tuning的比较结果。

这些笔记似乎用于学习或研究最新的人工智能技术和发展，特别是在自然语言处理和机器学习的领域。

![](./images/gpt-4v.jpg)

In [10]:
from IPython.display import display, Markdown
content = query_base64_image_description("./images/gpt-4v.jpg")
display(Markdown(content))

这张图片显示了一位粉丝在演唱会或者公共活动中举着一张写着文字的牌子。牌子上用中文写道：“康一张 祝你生日快乐 25岁生日快乐❤️”。从这段文字可以推测，这位粉丝正在庆祝某位名为“康一张”的人的25岁生日，表达了生日祝福和爱慕之情。 BACKGROUND: Field of Gait Analysis

The field of Gait Analysis involves the study of human locomotion, using the methodologies and technologies of biomechanics, neuroscience, mechanical engineering, and other disciplines to assess the ways people walk or run. This analysis can be fundamental in diagnosing health conditions like Alzheimer’s disease, multiple sclerosis, and cerebral palsy, as well as in sports science to enhance performance or prevent injuries.

Gait analysis typically involves measuring body movements, evaluating muscle activity, and in some scenarios, monitoring brain activity. Techniques can include video motion capture, force platforms that measure ground reaction forces, electromyography that measures muscle electrical activity, and goniometers that measure joint angles.

This multifacetal approach provides valuable data that can identify biomechanical abnormalities, assess how ailments affect gait, and help to devise personalized therapeutic strategies. For instance, in stroke rehabilitation, gait analysis helps determine the effectiveness of specific interventions and supports recovery by highlighting how to adjust walking patterns. In sports, athletes utilize gait analysis to optimize their efficiency and minimize risks of injury.

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。